### Check GPU availability

In [1]:
!nvidia-smi

Mon Dec 31 00:03:24 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   28C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 25%   38C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from tqdm import tqdm
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability

In [3]:
trivial.print_idst()
dstc2.check()

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!


### Retrieve data

In [4]:
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|       Dataset Retrieval        |
INFO:root:+--------------------------------+
INFO:root:Reading dstc2_train.flist, dstc2_dev.flist and ontology_dstc2.json
INFO:root:Asserted 1612 dialogs for dstc2_train.flist
INFO:root:Asserted 506 dialogs for dstc2_dev.flist
INFO:root:Extracting raw train features
100%|██████████| 1612/1612 [00:09<00:00, 176.92it/s]
INFO:root:Extracting raw dev features
100%|██████████| 506/506 [00:02<00:00, 211.68it/s]
INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features
100%|██████████| 1117/1117 [00:06<00:00, 182.43it/s]


### Set device

In [5]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|            Baseline            |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 1


### Create vocabularies

In [6]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm(raw_X_train):
    
    for raw_train_turn in raw_train_dialog["turns"]:
        
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries
100%|██████████| 3224/3224 [00:00<00:00, 37930.99it/s]


### Execution configuration

In [7]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
logging.info("VOCABULARY_SIZE:\t\t{}".format(VOCABULARY_SIZE))

# NOTE: +2 because of null and dontcare
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2
METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = len(ontology["requestable"])
EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100
NUM_EPOCHS = 30
GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()
logging.info("GOAL_FOOD_DIM:\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t{}".format(GOAL_AREA_DIM))
logging.info("METHOD_DIM:\t\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t\t{}".format(REQUESTED_DIM))
logging.info("EMBEDDING_DIM:\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_EPOCHS:\t\t\t{}".format(NUM_EPOCHS))
logging.info("GOAL_LOSS_FUNCTION:\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t{}".format(REQUESTED_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:		1149
INFO:root:GOAL_FOOD_DIM:		93
INFO:root:GOAL_PRICERANGE_DIM:		5
INFO:root:GOAL_NAME_DIM:		115
INFO:root:GOAL_AREA_DIM:		7
INFO:root:METHOD_DIM:			5
INFO:root:REQUESTED_DIM:		8
INFO:root:EMBEDDING_DIM:		170
INFO:root:ALTERED_EMBEDDING_DIM:	300
INFO:root:HIDDEN_DIM:			100
INFO:root:NUM_EPOCHS:			30
INFO:root:GOAL_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:	BCELoss()


### Models utilities

In [8]:
def get_index_and_score(turn, token_to_index, device):
    indices = []
    scores = []
    tokens_scores = turn["system"] + turn["user"]
    for token, score in tokens_scores:
        if token not in token_to_index:
            indices.append(token_to_index["<unk>"])
        else:
            indices.append(token_to_index[token])
        scores.append(score)
    assert len(indices) == len(scores)
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

def plotly_plot(train_accuracies, dev_accuracies, test_accuracies, accuracy_type):
    plotly.offline.iplot({
                            "data": [Scatter(
                                            x = list(range(len(train_accuracies))),
                                            y = train_accuracies,
                                            mode = "lines+markers",
                                            name = "Train {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#1abc9c")),
                                    Scatter(
                                            x = list(range(len(dev_accuracies))),
                                            y = dev_accuracies,
                                            mode = "lines+markers",
                                            name = "Dev {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#3498db")),
                                    Scatter(
                                            x = list(range(len(test_accuracies))),
                                            y = test_accuracies,
                                            mode = "lines+markers",
                                            name = "Test {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#9b59b6"))],
                            "layout": Layout(
                                             title = "<b>Train-Dev-Test {} Accuracy</b>".format(accuracy_type),
                                             xaxis = dict(title = "<b>Epoch</b>",
                                                          dtick = 1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))
                        })

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 1):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

get_bin = lambda x, n: format(x, "b").zfill(n)

def make_tracker(raw_X, raw_Y, dataset):
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_LecTrack.hidden = model_LecTrack.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, device = DEVICE)
                
                goal_foods, goal_priceranges, goal_names, goal_areas, methods, requesteds = model_LecTrack(indices, scores)
                
                goal_food = goal_foods[-1]
                goal_pricerange = goal_priceranges[-1]
                goal_name = goal_names[-1]
                goal_area = goal_areas[-1]
                method = methods[-1]
                requested = requesteds[-1]
                
                if torch.argmax(goal_food).item() != 0:
                    turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                if torch.argmax(goal_pricerange).item() != 0:
                    turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                if torch.argmax(goal_name).item() != 0:
                    turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                if torch.argmax(goal_area).item() != 0:
                    turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                
                session["turns"].append(turn)
            tracker_json["sessions"].append(session)
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

def get_scores(raw_X, raw_Y, dataset):
    goal_accuracy = 0
    requested_accuracy = 0
    method_accuracy = 0
    
    tracker_incremental = make_tracker(raw_X, raw_Y, dataset = dataset)
    
    with open("tracker_incremental.json", "w") as tracker_incremental_file:
        json.dump(tracker_incremental, tracker_incremental_file)
    
    if dataset == "dstc2_train":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_train\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_incremental.json\
        --scorefile tracker_incremental.score.csv
    elif dataset == "dstc2_dev":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_dev\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_incremental.json\
        --scorefile tracker_incremental.score.csv
    else:
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_test\
        --dataroot dstc2/dstc2_test/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_incremental.json\
        --scorefile tracker_incremental.score.csv

    file_cat = !python2 dstc2/dstc2_scripts/report.py --scorefile tracker_incremental.score.csv
    
    for line in file_cat:
        if line.startswith("Accuracy"):
            accuracies = line.split("|")
            goal_accuracy = float(accuracies[1])
            requested_accuracy = float(accuracies[2])
            method_accuracy = float(accuracies[3])
    
    return goal_accuracy, requested_accuracy, method_accuracy

### LecTrack

In [9]:
class LecTrackModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_food_dim,
                 goal_pricerange_dim,
                 goal_name_dim,
                 goal_area_dim,
                 method_dim,
                 requested_dim,
                 device):
        
        super(LecTrackModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_food_dim = goal_food_dim
        self.goal_pricerange_dim = goal_pricerange_dim
        self.goal_name_dim = goal_name_dim
        self.goal_area_dim = goal_area_dim
        self.method_dim = method_dim
        self.requested_dim = requested_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_food_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_food_dim)
        
        self.goal_pricerange_classifier = nn.Linear(in_features = hidden_dim,
                                                    out_features = goal_pricerange_dim)
        
        self.goal_name_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_name_dim)
        
        self.goal_area_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_area_dim)
        
        self.method_classifier = nn.Linear(in_features = hidden_dim,
                                           out_features = method_dim)
        
        self.requested_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = requested_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_food = F.log_softmax(self.goal_food_classifier(lstm_out).view(-1, self.goal_food_dim), dim = 1)        
        
        goal_pricerange = F.log_softmax(self.goal_pricerange_classifier(lstm_out).view(-1, self.goal_pricerange_dim), dim = 1) 
        
        goal_name = F.log_softmax(self.goal_name_classifier(lstm_out).view(-1, self.goal_name_dim), dim = 1)
        
        goal_area = F.log_softmax(self.goal_area_classifier(lstm_out).view(-1, self.goal_area_dim), dim = 1)
        
        method = F.log_softmax(self.method_classifier(lstm_out).view(-1, self.method_dim), dim = 1)
        
        requested = torch.sigmoid(self.requested_classifier(lstm_out).view(-1, self.requested_dim))
        
        return goal_food, goal_pricerange, goal_name, goal_area, method, requested

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        goal_food_dict["dontcare"] = output_tensor[1].item() 
        for index in range(len(output_tensor) - 2):
            goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

def retrieve_gold_GoalPriceRange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        goal_pricerange_dict["dontcare"] = output_tensor[1].item()
        for index in range(len(output_tensor) - 2):     
            goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        goal_name_dict["dontcare"] = output_tensor[1].item()
        for index in range(len(output_tensor) - 2):
            goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
        
    return goal_name_dict

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    
    index = torch.argmax(output_tensor).item()
    if index != 0:
        goal_area_dict["dontcare"] = output_tensor[1].item()
        for index in range(len(output_tensor) - 2):
            goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
        
    return goal_area_dict

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    
    return method_dict

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        if np.greater_equal(probability_value, 0.5):
            requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

model_LecTrack = LecTrackModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               method_dim = METHOD_DIM,
                               requested_dim = REQUESTED_DIM,
                               device = DEVICE)

model_LecTrack = model_LecTrack.to(DEVICE)

optimizer_LecTrackModel = optim.Adam(model_LecTrack.parameters(), lr = 1e-4)

### Train LecTrack

In [10]:
train_global_accuracies = []
train_requested_accuracies = []
train_method_accuracies = []

dev_global_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_global_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

lectrack_early_stopping = EarlyStopping(patience = 5,
                                        min_delta = 0.005)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_LecTrack = model_LecTrack.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm(zip(raw_X_train, raw_Y_train), total = len(raw_X_train)):

        model_LecTrack.hidden = model_LecTrack.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_LecTrackModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, device = DEVICE)
        
            goal_foods, goal_priceranges, goal_names, goal_areas, methods, requesteds = model_LecTrack(indices, scores)
            
            for goal_food, goal_pricerange, goal_name, goal_area, method, requested in zip(goal_foods, goal_priceranges, goal_names, goal_areas, methods, requesteds):
                
                goal_food = goal_food.unsqueeze(0)
                goal_pricerange = goal_pricerange.unsqueeze(0)
                goal_name = goal_name.unsqueeze(0)
                goal_area = goal_area.unsqueeze(0)
                method = method.unsqueeze(0)
                requested = requested.unsqueeze(0)
                
                loss_goal_food = GOAL_LOSS_FUNCTION(goal_food,
                                                    retrieve_gold_GoalFood(raw_Y_train_turn,
                                                                           ontology = ontology,
                                                                           device = DEVICE))

                loss_goal_pricerange = GOAL_LOSS_FUNCTION(goal_pricerange,
                                                          retrieve_gold_GoalPriceRange(raw_Y_train_turn,
                                                                                       ontology = ontology,
                                                                                       device = DEVICE))

                loss_goal_name = GOAL_LOSS_FUNCTION(goal_name,
                                                    retrieve_gold_GoalName(raw_Y_train_turn,
                                                                           ontology = ontology,
                                                                           device = DEVICE))

                loss_goal_area = GOAL_LOSS_FUNCTION(goal_area,
                                                    retrieve_gold_GoalArea(raw_Y_train_turn,
                                                                           ontology = ontology,
                                                                           device = DEVICE))

                loss_method = METHOD_LOSS_FUNCTION(method,
                                                   retrieve_gold_Method(raw_Y_train_turn,
                                                                        ontology = ontology,
                                                                        device = DEVICE))

                loss_requested = REQUESTED_LOSS_FUNCTION(requested,
                                                         retrieve_gold_Requested(raw_Y_train_turn,
                                                                                 ontology = ontology,
                                                                                 device = DEVICE))

                loss = loss_goal_food + loss_goal_pricerange + loss_goal_name + loss_goal_area + loss_method + loss_requested
                loss.backward(retain_graph = True)

                optimizer_LecTrackModel.step()
    
    model_LecTrack = model_LecTrack.eval()
    
    with torch.no_grad():
        
        train_global_accuracy, \
        train_requested_accuracy, \
        train_method_accuracy = get_scores(raw_X_train, raw_Y_train, dataset = "dstc2_train")
        logging.info("TRAIN:\t{}\t{}\t{}".format(train_global_accuracy, train_requested_accuracy, train_method_accuracy))
        train_global_accuracies.append(train_global_accuracy)
        train_requested_accuracies.append(train_requested_accuracy)
        train_method_accuracies.append(train_method_accuracy)
        
        dev_global_accuracy, \
        dev_requested_accuracy, \
        dev_method_accuracy = get_scores(raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
        logging.info("DEV:\t\t{}\t\t{}\t\t{}".format(dev_global_accuracy, dev_requested_accuracy, dev_method_accuracy))
        dev_global_accuracies.append(dev_global_accuracy)
        dev_requested_accuracies.append(dev_requested_accuracy)
        dev_method_accuracies.append(dev_method_accuracy)
        
        test_global_accuracy, \
        test_requested_accuracy, \
        test_method_accuracy = get_scores(raw_X_test, raw_Y_test, dataset = "dstc2_test")
        logging.info("TEST:\t\t{}\t\t{}\t\t{}".format(test_global_accuracy, test_requested_accuracy, test_method_accuracy))
        test_global_accuracies.append(test_global_accuracy)
        test_requested_accuracies.append(test_requested_accuracy)
        test_method_accuracies.append(test_requested_accuracy)
        
    lectrack_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = dev_global_accuracy)
    
    if lectrack_early_stopping.stop_training:
        break

plotly_plot(train_global_accuracies, dev_global_accuracies, test_global_accuracies, "Global")
plotly_plot(train_requested_accuracies, dev_requested_accuracies, test_requested_accuracies, "Requested")
plotly_plot(train_method_accuracies, dev_method_accuracies, test_method_accuracies, "Method")

INFO:root:Epoch	1/30
100%|██████████| 3224/3224 [00:55<00:00, 58.23it/s]
INFO:root:TRAIN:	0.0104344	0.4553571	0.5948286
100%|██████████| 506/506 [00:09<00:00, 54.12it/s]
INFO:root:DEV:		0.214025		0.6004327		0.6852043
100%|██████████| 1117/1117 [00:23<00:00, 46.72it/s]
INFO:root:TEST:		0.2706162		0.5815246		0.7037151
INFO:root:Epoch	2/30
100%|██████████| 3224/3224 [00:50<00:00, 63.57it/s]
INFO:root:TRAIN:	0.0085063	0.4538328	0.6468803
100%|██████████| 506/506 [00:08<00:00, 58.60it/s]
INFO:root:DEV:		0.1634515		0.5975478		0.7578893
100%|██████████| 1117/1117 [00:22<00:00, 49.45it/s]
INFO:root:TEST:		0.1811332		0.5782254		0.8067529
INFO:root:Epoch	3/30
100%|██████████| 3224/3224 [00:52<00:00, 61.51it/s]
INFO:root:TRAIN:	0.0139503	0.4736498	0.6321529
100%|██████████| 506/506 [00:09<00:00, 56.13it/s]
INFO:root:DEV:		0.2244526		0.6393797		0.8041904
100%|██████████| 1117/1117 [00:22<00:00, 48.76it/s]
INFO:root:TEST:		0.2823821		0.6414308		0.78828
INFO:root:Epoch	4/30
100%|██████████| 3224/322